### Load libraries and external data

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import colored
from colored import stylize
from data import dataloader
import datetime
import json
from neuralnet import nn
import numpy as np
import pandas as pd
#import pixiedust_node #v≥0.2.5

#### Load data from Firebase.
Requires [Firebase service account credentials](https://console.firebase.google.com/project/tingle-pilot-collected-data/settings/serviceaccounts/adminsdk) in JSON format saved in `./firebase-credentials`.

In [9]:
pilot_data, notes = dataloader.load_from_firebase(
    notes=True,
    start=datetime.datetime(2018,3,6,8),
    combine=True,
    marked=True
)

The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.
Data loaded from Firebase!


#### Load [Synaptic](http://caza.la/synaptic/)
If "Error: Cannot find module 'synaptic'", create and run these two cells:

1. ```
cd neuralnet
```

2. ```sh
!npm init -y
!npm install -s synaptic
```

In [ ]:
%%node
var lstm = require('../../tingle-pilot-study/neuralnet/lstm.js');

---
### See all targets, number of available samples and iteration blocks

In [ ]:
for target in list(pilot_data.target.unique()):
    ib =max(
        pilot_data.loc[
            pilot_data.target==target
        ].iteration_block.dropna()
    )
    print(": ".join([
        target,
        "{0} on-target samples in {1} iteration block{2}".format(
            str(len(pilot_data.loc[
                (pilot_data.target == target) &
                (pilot_data.ontarget)
            ])),
            "%.0f" % ib,
            "s" if ib != 1 else ""
        )
    ]))

---
### Extract training and testing data
Define targets of interest and corresponding offtargets

In [ ]:
with open(
    'data/targets.json',
    'r'
) as fp:
    targets = json.load(
        fp
    )[0]

Set parameters for nn:

In [ ]:
input_signals = [
    "distance",
    "thermopile1",
    "thermopile2",
    "thermopile3",
    "thermopile4"
]
iteration_blocks = list(range(1,7))
n_samples = 100

Get training inputs and outputs, inputs that should evaluate ~true and inputs that should evaluate ~false

In [ ]:
data = {}
for target in targets:
    data[target] = {
        "train": nn.define_trainer_data(
            pilot_data,
            {
                "target": [target],
                "offtarget": targets[target]
            },
            input_signals,
            iteration_blocks,
            n_samples
        ),
        "test_true": nn.define_activation(
            pilot_data,
            [target],
            input_signals,
            iteration_blocks,
            exclude=n_samples
        ),
        "test_false": nn.define_activation(
            pilot_data,
            targets[target],
            input_signals,
            iteration_blocks,
            exclude=n_samples
        )
    }

Preview all inputs and training outputs

In [ ]:
data

**Note**: These data take some time to copy across environments. Give the notebook some time between running cells across Python and JavaScript.

---
### Train and test

In [ ]:
%%node
var networks = {};
for (var target in data) {
  networks[target] = lstm.train_lstm([5,5,2,1], data[target]["train"], 0.06, 0.06, 3000);
}

In [ ]:
%%node
var test_outputs = {};
for (var target in data) {
    test_outputs[target] = {"true":[],"false":[]};
    for(var iteration=0; iteration < data[target]["test_true"].length; iteration++){
      test_outputs[target]["true"].push(networks[target].activate(data[target]["test_true"][iteration]));
      }
    for(var iteration=0; iteration < data[target]["test_false"].length; iteration++){
      test_outputs[target]["false"].push(networks[target].activate(data[target]["test_false"][iteration]));
    }
}

In [ ]:
test_outputs

---
### See outputs

If the training is adequate x ≈ 0 ∀ x in the following:

In [ ]:
f = {target: [
        outputs for outputs in test_outputs[target]['false']
] for target in targets}
f

If the training is adequate x ≈ 1 ∀ x in the following:

In [ ]:
t = {target: [
        outputs for outputs in test_outputs[target]['true']
] for target in targets}
t

Finally, if training is adequate, f ≪ t:

In [ ]:
f_mean = {
    target: np.mean(f[target]) for target in targets if target in f and len(f[target])
}
t_mean = {
    target: np.mean(t[target]) for target in targets if target in t and len(t[target])
}
for target in t_mean:
    print(target)
    print(
        "f = {0}\nt = {1}\n{0} ≪ {1} ?\n".format(
            str(f_mean[target]),
            str(t_mean[target])
        ) if f_mean[target] < t_mean[target] else "f = {0}\nt = {1}\n{2}".format(
            str(f_mean[target]),
            str(t_mean[target]),
            stylize(
                "Nope. f > t\n",
                colored.fg("red")
            )
        )
    )
for target in t_mean:
    print("{0}: f = {1:.4f} < t = {2:.4f}".format(
        target,
        f_mean[target],
        t_mean[target]
    ))

---
### Try with demo data

In [ ]:
with open(
    'data/demo_data.json',
    'r'
) as fp:
    demo_data = json.load(
        fp
    )[0]

In [ ]:
scale = list(
    pd.DataFrame(
        [
            a[0:5] for a in [
                *demo_data["true"],
                *demo_data["false"]
            ]
        ]
    ).astype('float').max(axis=0)
)
demo_test = {
    "train":[
        *[{
            "input": [
                float(d)/scale[datum_i] for datum_i, d in enumerate(datum[0:5])
            ],
            "output": [1]
        } for datum in demo_data["true"][0:40]],
        *[{
            "input": [
                float(d)/scale[datum_i] for datum_i, d in enumerate(datum[0:5])
            ],
            "output": [0]
        } for datum in demo_data["false"][0:40]]
    ],
    "test_true": [[
                float(d)/scale[datum_i] for datum_i, d in enumerate(datum[0:5])
            ] for datum in demo_data["true"][41:]],
    "test_false": [[
                float(d)/scale[datum_i] for datum_i, d in enumerate(datum[0:5])
            ] for datum in demo_data["false"][41:]]
}

In [ ]:
%%node
var LSTM5 = lstm.train_lstm([5,5,2,1], demo_test["train"], 0.06, 0.06, 3000);

In [ ]:
%%node
var demo_test_outputs = {"true":[],"false":[]}
for(var iteration=0; iteration < demo_test["test_true"].length; iteration++){
    var op = LSTM5.activate(demo_test["test_true"][iteration]);
    demo_test_outputs["true"].push(op);
}
for(var iteration=0; iteration < demo_test["test_false"].length; iteration++){
    demo_test_outputs["false"].push(LSTM5.activate(demo_test["test_false"][iteration]));
}

In [ ]:
demo_test_outputs

In [ ]:
np.mean(demo_test_outputs['false'])

In [ ]:
np.mean(demo_test_outputs['true'])